In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import stats
import os

os.chdir("C:/Users/mailm/OneDrive - Université paris nanterre/Documents/CMI/L3 S1/Atelier CMI/Calcul Numérique/Projet")

# Election présidentielle 2022 et variables socio-démographiques dans les Hauts-de-Seine

données :

- données elections présidentielles 2022
- Variables socio demographiques 92
- Données département fr

In [2]:
# Chargement des données

pres22 = pd.read_csv("C:/Users/mailm/OneDrive - Université paris nanterre/Documents/CMI/L3 S1/Atelier CMI/Calcul Numérique/Projet/data/pres2022comm.csv", dtype={'codecommune': str, 'dep': str})
